%md
## Publish a Data Product
In this notebook we want to publish the cashflow prediction results to the data catalog of SAP Business Data Cloud. For that we need to build a custom data product by using the python library [sap-bdc-connect-sdk](https://pypi.org/project/sap-bdc-connect-sdk/). We will also utilize the [Delta Share protocol](https://delta.io/sharing/), which allows us to share the data product without the need of copying the result table. The result table remains persisted in SAP Databricks, and will be remotely accessible from the data catalog. 

The following steps need to be applied in this exercise:

1. Install and load packages
2. Create a client for sap-bdc-connect-sdk
3. Create Delta Share
4. Add Recipient to Share
5. Create Open Resource Discovery (ORD) object
6. Create Core Schema Notation (CSN) object
7. Publish data product

%md
> **Pre-requisite: SECRET SCOPE**
>
> This step has already been done by the administrator to facilitate the sharing process. However, if you need to do > it on you own you can follow the steps described here to create a secret scope:
>
> To create a secret scope you can either use the following URL https://<databricks-instance>#secrets/createScope. 
> Replace <databricks-instance> with the workspace URL of your Databricks deployment.
>
> Alternatively, you can run the following command in the terminal by clicking on the terminal icon on the lower  right corner: `databricks secrets create-scope sap-bdc-connect-sdk`. 
>
>The secret scope only has to be created once and can be made accessible to all workspace users by either toggling  `manage principal` to `all workspace users` or via the terminal using the following command `databricks secrets put-acl sap-bdc-connect-sdk users READ`. To check whether the assignment worked, you can then use the command `databricks secrets list-acls sap-bdc-connect-sdk`.
>
> A full explanation can be found here https://docs.databricks.com/aws/en/security/secrets/example-secret-workflow

### 1. Install and load Packages
To be able to share the enhanced data products back to SAP Business Data Cloud, we need the `SAP SDK` (https://pypi.org/project/sap-bdc-connect-sdk/) 

In [0]:
%pip install sap-bdc-connect-sdk
%pip install --upgrade pydantic
%restart_python

Check the sdk details and version.

In [0]:
pip show sap-bdc-connect-sdk

### 2. Create a client for sap-bdc-connect-sdk
The `DatabricksClient` receives dbutils as a parameter, which is a Databricks utility that can be used inside the Databricks notebooks.

The `BdcConnectClient` receives the DatabricksClient as a parameter to get information from the Databricks environment (e.g. secrets, api_token, workspace_url_base)

In [0]:
from bdc_connect_sdk.auth import BdcConnectClient
from bdc_connect_sdk.auth import DatabricksClient
from bdc_connect_sdk.utils import csn_generator
import json

databricks_client = DatabricksClient(dbutils)
bdc_connect_client = BdcConnectClient(databricks_client)

### 3. Create Delta Share
A share is a mechanism for distributing and accessing data across different systems. Creating or updating a share involves including specific attributes, such as @openResourceDiscoveryV1, in the request body, aligning with the Open Resource Discovery protocol. This procedure ensures that the share is properly structured and described according to specified standards, facilitating effective data sharing and management. This is done in step 5.

Please replace the values `<CATALOG_NAME>` and `<SCHEMA_NAME>` with the specific values that match our use case and group. You can find the correct names by checking the **Unity Catalog** and look for the specific catalog and schema names: `uc_XXX`, `grpX`.

In [0]:
%sql
-- CREATE CATALOG IF NOT EXISTS <CATALOG_NAME>;
SET CATALOG uc_cash_liquidity_forecast;
CREATE SCHEMA IF NOT EXISTS grp01;
USE SCHEMA grp01;

Now let's create the Delta Share and include the two results tables created in the schema into the Delta Share. 

Please replace variable `<SHARE_NAME>` with the value `grpX_share_cash_liquidity`, and the variable `<PREDICTION_RESULT_TABLE_NAME>` with the corresponding table names of the Unity Catalog.

In [0]:
%sql
CREATE SHARE IF NOT EXISTS grp01_share_cash_liquidity;
ALTER SHARE grp01_share_cash_liquidity ADD TABLE cashflow_prediction WITH HISTORY;

### 4. Add Recipient to Share

To be able to share the data product to BDC we need to select `sap-business-data-cloud` as the recipient for the share we've just created. 

The recipient can be added using the following query. Please fill in the variable `<RECIPIENT_NAME>` with the value `sap-business-data-cloud`:

In [0]:
%sql
GRANT SELECT ON SHARE grp01_share_cash_liquidity TO RECIPIENT `sap-business-data-cloud`;

%md
Remark: Alternatively, this can be done by heading over to the **Unity Catalog** > **Delta Sharing** > **Shared by me** > *Select Share* > **Recipient** > **Add Recipients** > *Select sap-business-data-cloud*

### 5. Create Open Resource Discovery (ORD) object 

%md
Creating or updating a share involves including specific attributes, such as @openResourceDiscoveryV1, in the request body, aligning with the [Open Resource Discovery protocol](https://open-resource-discovery.github.io/specification/). This procedure ensures that the share is properly structured and described according to specified standards, facilitating effective data sharing and management. This is done in step 5.

In the following code you'll need to set the parameters for the share:
- `<DATA_PRODUCT_NAME>` : 
    - e.g. "grpxx Cashflow Prediction"
- `<SHORT_DESCRIPTION>` : 
    - e.g. "grpxx Data Produt for Cashflow Prediction"
- `<LONG_DESCRIPTION>`: 
    - e.g. "This data product contains the data used for the cashflow prediction model. A hyperparameter optimization is performed on the cleased data with the help of the Bayesian Search optimization. For the clustering we use the Affinity Propagation algorithm. After the hyperparameter optimization is performed, the model with the highest silhouette score is applied as the best model to the prepared clustering dataset. The resulting clustering labels are stored and merged together with a T-SNE based representation of the input dataset."

In [0]:
from bdc_connect_sdk.auth import BdcConnectClient
from bdc_connect_sdk.auth import DatabricksClient

bdc_connect_client = BdcConnectClient(DatabricksClient(dbutils, "sap-business-data-cloud"))

share_name = "grp01_share_cash_liquidity"

open_resource_discovery_information = {
    "@openResourceDiscoveryV1": {
        "title": "grp01 Cashflow Predictions Data Product",
        "shortDescription": "Data asset for cashflow predictions",
        "description": "This data product contains the data used for the cashflow prediction model. A hyperparameter optimization is performed on the cleased data with the help of the Bayesian Search optimization. For the clustering we use the Affinity Propagation algorithm. After the hyperparameter optimization is performed, the model with the highest silhouette score is applied as the best model to the prepared clustering dataset. The resulting clustering labels are stored and merged together with a T-SNE based representation of the input dataset."
    }
}

response = bdc_connect_client.create_or_update_share(
    share_name,
    open_resource_discovery_information
)
print(f"[REQUEST] Create or update share request was executed and returned {response}")

### 6. Create Core Schema Notation (CSN) object

The Core Schema Notation (CSN serves as a standardized format for configuring and describing shares within a network. To create or update the CSN for a share, it's advised to prepare the CSN content in a separate file and include this content in the request body. This approach ensures accuracy and compliance with the CSN interoperability specifications, facilitating consistent and effective share configuration across systems.

In the following code you'll need to set the parameters `<RECIPIENT_NAME>` and `<SHARE_NAME>` for the share.

In [0]:
from bdc_connect_sdk.auth import BdcConnectClient
from bdc_connect_sdk.auth import DatabricksClient
from bdc_connect_sdk.utils import csn_generator

bdc_connect_client = BdcConnectClient(DatabricksClient(dbutils, "sap-business-data-cloud"))

share_name = "grp01_share_cash_liquidity"

csn_schema = csn_generator.generate_csn_template(share_name)

response = bdc_connect_client.create_or_update_share_csn(
    share_name,
    csn_schema
)
print(f"[REQUEST] Create or update CSN request was executed and returned {response if response else 'OK'}")

### 7. Publish a Data Product
A Data Product is an abstraction that represents a type of data or data set within a system, facilitating easier management and sharing across different platforms. It bundles resources or API endpoints to enable efficient data access and utilization by integrated systems. Publishing a Data Product allows these systems to access and consume the data, ensuring seamless communication and resource sharing.

In the following code you'll need to set the parameters `<RECIPIENT_NAME>` and `<SHARE_NAME>` for the share.

In [0]:
from bdc_connect_sdk.auth import BdcConnectClient
from bdc_connect_sdk.auth import DatabricksClient

bdc_connect_client = BdcConnectClient(DatabricksClient(dbutils, "sap-business-data-cloud"))

share_name = "grp01_share_cash_liquidity"

response = bdc_connect_client.publish_data_product(
    share_name
)
print(f"[REQUEST] Publish Data Product request was executed and returned {response if response else 'OK'}")